# SPICAV Exploratory Data Analysis

## This notebook explores the SPICAV dataset from the Venus Express satellite

## Author: g.mahapatra@tudelft.nl

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# set the path to SPICAV data
path = ''